In [19]:
import pandas as pd
import json

In [20]:
pd.set_option('display.max_columns', 999)

### JSON QUERIES

In [21]:
with open(r'C:\Users\DANIEL\workspace\gw2_srs\GW2_SRS\ETL\EXTRACT_00\Web Scraping\Boss_data\Wing_1\Valley_Guardian\20220823-171406_vg_kill.json') as file:
    data = json.load(file)

# It is possible to use json.loads(file.read()) to convert the JSON into a Python dictionary
# Or just json.load(file) to read the file

# Difference betwwen json.load and json loads:
    # json.load will take a file object, i.e: /file/data.json
    # json.loads will take a string, i.e: {"key": 1, "name": red}
    # In case of using loads with a file, we must use json.loads(file.read()) so we can read the content

In [22]:
type(data)

dict

#### Data to find:
- Players:
    - Name: data['players'][0]['name']
---
- DPS:
    - Per Phase:
        - *Phase 1*: data['phases'][1]
            - *Phase1_dps*: data['phases'][1]['dpsStats'][0:9]
                - Pos.0 is both Power and Condi damage
                - We need to create a variable to read the dps position
        - *Phase 2*: data['phases'][6]
            - *Phase2_dps*: data['phases'][6]['dpsStats'][0:9]
        - *Phase 3*: data['phases'][12]
            - *Phase3_dps*: data['phases'][12]['dpsStats'][0:9]
---
- Phase Duration:
    - *Phase 1*: data['phases'][1]['duration']
        - all should be rounded as they all come as full 5 digits, i.e: 37842 = 37,8s
    - *Phase 2*: data['phases'][6]['duration']
    - *Phase 3*: data['phases'][12]['duration']

In [23]:
print(data['players'][0]['name'])

print('Total DPS:', data['players'][0]['details']['dmgDistributions'][0]['totalDamage'])
# We can access the information from the actual JSON by using queries

Paiani
Total DPS: 2484019


In [24]:
players_name = data['players'][0:10]
print(f'{"Players_Name":<10}{"Total_DPS":>15}')
print(f'{"-"*len("Players_Name"):<10}{"-"*len("Total_DPS"):>15}')
for player in players_name:
    players = []
    dps_list = []

    names = player["name"]
    dps_stats = player["details"]["dmgDistributions"][0]["totalDamage"]

    # for name in names:
    #     players.append(name)
    # for dps in dps_stats:
    #     dps_list.append(dps)

    print(f'{names:<10}{dps_stats:>16}')

# print(players)
# print(dps_list)

Players_Name      Total_DPS
------------      ---------
Paiani             2484019
Xxkakarot          3399400
Headshot Free          429369
Saiku Goldmane         1833554
Deutschlad         3034161
Isadora Sunspear          605640
F L O R I Ä N         5174108
Exxcelsiorr         4422165
Nanacia            2755294
Carvalhoooo         2538895


In [25]:
dps_all = data['phases'][1]['dpsStats'][0]
print(dps_all[0])
print('-'*10)
for dps_data in dps_all:
    print(dps_data)

787747
----------
787747
113461
674286
0.0


In [26]:
def get_dps_data():
    
    full_fight = data['phases'][0]['dpsStats'][0:10]
    phase1_dps = data['phases'][1]['dpsStats'][0:10]
    phase2_dps = data['phases'][6]['dpsStats'][0:10]
    phase3_dps = data['phases'][12]['dpsStats'][0:10]

    space0 = ' '*9
    space1 = ' '*6
    space2 = ' '*5

    #print(f"{'Full_Fight_DPS'}{' '*2}{'Phase_1_DPS'}{' '*2}{'Phase_2_DPS'}{' '*2}{'Phase_3_DPS'}")
    #print(f"{'-'*len('Full_Fight_DPS')}{' '*2}{'-'*len('Phase_1_DPS')}{' '*2}{'-'*len('Phase_2_DPS')}{' '*2}{'-'*len('Phase_3_DPS')}")

    for dps0,dps1,dps2,dps3 in zip(full_fight,phase1_dps,phase2_dps,phase3_dps):
        # It's possible to store multiple data with zip in a for loop
        print(dps0[0],space0,dps1[0],space1,dps2[0],space2,dps3[0])
pass

get_dps_data()

2484019           787747        540528       855179
3399400           994036        938230       1002514
429369           100178        141115       113644
1833554           662356        558936       458636
3034161           840520        861832       1014995
605640           154467        142031       193136
5174108           1361691        1379999       1631317
4422165           1109025        1181622       1466977
2755294           775455        786923       840366
2538895           703522        691528       721951


In [27]:
print('Battle Duration:', data['encounterDuration']) # 194s
# https://www.inchcalculator.com/time-to-seconds-calculator/

dps = data['phases'][0]['dpsStats'][0][1]
dps

Battle Duration: 03m 14s 414ms


413467

In [28]:
phase1_time_raw = data['phases'][1]['duration']
phase1_time = round(phase1_time_raw/1000,1)

phase2_time_raw = data['phases'][6]['duration']
phase2_time = round(phase2_time_raw/1000,1)

phase3_time_raw = data['phases'][12]['duration']
phase3_time = round(phase3_time_raw/1000,1)

total_time = phase1_time+phase2_time+phase3_time

print(f"{phase1_time}s {phase2_time}s {phase3_time}s")
print(f"{total_time}s")

37.8s 39.1s 39.1s
116.0s


In [29]:
rd = round(dps/194,2) # We divide the total dps by the time of the encounter in seconds
rd 

2131.27

### JSON DATAFRAME NORMALIZED

In [38]:
df = pd.json_normalize(data, max_level=None, record_path=['players']) # meta=['dmgDistributions', 'dmgDistributionsTargets'], errors='ignore'
# Meta let us specify which data we want in columns apart of the record_path
# errors='ignore' let us avoid worrying about NaNs
# We can also set our personal separators with sep='->'
# We can as well set prefixes:
    # meta_prefix='meta-'
    # record_prefix='player-'

In [35]:
df

,group,acc,profession,isPoV,isCommander,l1Set,l2Set,a1Set,a2Set,colTarget,colCleave,colTotal,isFake,notInSquad,uniqueID,name,tough,condi,conc,heal,icon,health,minions,details.dmgDistributions,details.dmgDistributionsTargets,details.dmgDistributionsTaken,details.rotation,details.boonGraph,details.food,details.minions
0,2,Lemos.8360,Scourge,False,False,"[Scepter, Torch]","[Scepter, Warhorn]",[],[],"rgb(82,167,111)","rgb(64,127,85)","rgb(46,88,60)",False,False,4,Paiani,0,10,0,0,https://wiki.guildwars2.com/images/0/06/Scourg...,-1,"[{'id': 48, 'name': 'Shadow Fiend'}]","[{'contributedDamage': 2428840, 'contributedBr...","[[{'contributedDamage': 1903033, 'contributedB...","[{'contributedDamage': 126637, 'contributedBre...","[[[1.069, 10698, 1010, 1, -0.002], [2.079, 106...","[[{'id': -18, 'color': '', 'visible': False, '...","[{'time': 0.0, 'duration': 3578.752, 'id': 571...",[{'dmgDistributions': [{'contributedDamage': 5...
1,2,Brando.1374,Specter,True,False,"[Scepter, Dagger]",[],[],[],"rgb(192,143,149)","rgb(146,109,114)","rgb(101,76,79)",False,False,34,Xxkakarot,0,10,0,0,https://wiki.guildwars2.com/images/5/5c/Specte...,-1,"[{'id': 125, 'name': 'Thief'}, {'id': 126, 'na...","[{'contributedDamage': 3305345, 'contributedBr...","[[{'contributedDamage': 2766494, 'contributedB...","[{'contributedDamage': 87735, 'contributedBrea...","[[[1.88, 13082, 670, 2, -0.002], [2.55, 23275,...","[[{'id': -18, 'color': '', 'visible': False, '...","[{'time': 0.0, 'duration': 1506.841, 'id': 651...",[{'dmgDistributions': [{'contributedDamage': 4...
2,2,lPRATESl.6931,Firebrand,False,True,"[Axe, Shield]",[Staff],[],[],"rgb(114,193,217)","rgb(88,147,165)","rgb(62,101,113)",False,False,45,Headshot Free,10,0,10,10,https://wiki.guildwars2.com/images/0/02/Firebr...,-1,"[{'id': 128, 'name': 'Bow of Truth'}]","[{'contributedDamage': 429369, 'contributedBre...","[[{'contributedDamage': 329231, 'contributedBr...","[{'contributedDamage': 119269, 'contributedBre...","[[[-0.683, 44364, 1237, 1, 0.739], [0.554, 400...","[[{'id': -18, 'color': '', 'visible': False, '...","[{'time': 0.0, 'duration': 3571.908, 'id': 572...",[{'dmgDistributions': [{'contributedDamage': 0...
3,2,Andarck.5610,Renegade,False,False,[Shortbow],"[Mace, Axe]",[],[],"rgb(209,110,90)","rgb(159,85,70)","rgb(110,60,50)",False,False,51,Saiku Goldmane,0,10,0,0,https://wiki.guildwars2.com/images/7/7c/Renega...,-1,"[{'id': 123, 'name': 'Jas Razorclaw'}, {'id': ...","[{'contributedDamage': 1833554, 'contributedBr...","[[{'contributedDamage': 1632661, 'contributedB...","[{'contributedDamage': 99459, 'contributedBrea...","[[[1.23, 27066, 561, 1, -0.006], [1.791, 27066...","[[{'id': -18, 'color': '', 'visible': False, '...","[{'time': 0.0, 'duration': 3564.75, 'id': 5712...",[{'dmgDistributions': [{'contributedDamage': 0...
4,2,The Optional.1538,Scourge,False,False,"[Scepter, Warhorn]","[Scepter, Torch]",[],[],"rgb(82,167,111)","rgb(64,127,85)","rgb(46,88,60)",False,False,86,Deutschlad,0,10,0,0,https://wiki.guildwars2.com/images/0/06/Scourg...,-1,"[{'id': 85, 'name': 'Shadow Fiend'}, {'id': 11...","[{'contributedDamage': 2970172, 'contributedBr...","[[{'contributedDamage': 2465464, 'contributedB...","[{'contributedDamage': 82137, 'contributedBrea...","[[[0.316, 44946, 753, 3, 0.016], [1.272, 10698...","[[{'id': -18, 'color': '', 'visible': False, '...","[{'time': 0.0, 'duration': 3568.397, 'id': 571...",[{'dmgDistributions': [{'contributedDamage': 4...
5,3,Saiki.5147,Druid,False,False,"[Axe, Warhorn]",[Staff],[],[],"rgb(140,220,130)","rgb(107,167,100)","rgb(75,115,70)",False,False,3,Isadora Sunspear,0,0,10,10,https://wiki.guildwars2.com/images/d/d2/Druid_...,-1,"[{'id': 2, 'name': 'Juvenile Electric Wyvern'}...","[{'contributedDamage': 379429, 'contributedBre...","[[{'contributedDamage': 244034, 'contributedBr...","[{'contributedDamage': 102831, 'contributedBre...","[[[-0.61, 31710, 1364, 1, -0.006], [0.754, 317...","[[{'id': -18, 'color': '', 'visible': False, '...","[{'time': 0.0, 'duration': 3569.4

In [29]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 30 columns):
 #   Column                           Non-Null Count  Dtype 
---  ------                           --------------  ----- 
 0   group                            10 non-null     int64 
 1   acc                              10 non-null     object
 2   profession                       10 non-null     object
 3   isPoV                            10 non-null     bool  
 4   isCommander                      10 non-null     bool  
 5   l1Set                            10 non-null     object
 6   l2Set                            10 non-null     object
 7   a1Set                            10 non-null     object
 8   a2Set                            10 non-null     object
 9   colTarget                        10 non-null     object
 10  colCleave                        10 non-null     object
 11  colTotal                         10 non-null     object
 12  isFake                           10 non

In [33]:
df.value_counts('profession')

profession
Firebrand    3
Mechanist    2
Scourge      2
Druid        1
Renegade     1
Specter      1
dtype: int64

In [30]:
columns = df.columns
columns

Index(['group', 'acc', 'profession', 'isPoV', 'isCommander', 'l1Set', 'l2Set',
       'a1Set', 'a2Set', 'colTarget', 'colCleave', 'colTotal', 'isFake',
       'notInSquad', 'uniqueID', 'name', 'tough', 'condi', 'conc', 'heal',
       'icon', 'health', 'minions', 'details.dmgDistributions',
       'details.dmgDistributionsTargets', 'details.dmgDistributionsTaken',
       'details.rotation', 'details.boonGraph', 'details.food',
       'details.minions'],
      dtype='object')

In [31]:
drop = df.drop(['isPoV', 'isCommander', 'l1Set', 'l2Set',
    'a1Set', 'a2Set', 'colTarget', 'colCleave', 'colTotal', 'isFake',
    'notInSquad', 'details.rotation', 'details.boonGraph', 'details.food',
    'details.minions', 'icon', 'minions', 'health'], axis=1)
drop

,group,acc,profession,uniqueID,name,tough,condi,conc,heal,details.dmgDistributions,details.dmgDistributionsTargets,details.dmgDistributionsTaken
0,2,Lemos.8360,Scourge,4,Paiani,0,10,0,0,"[{'contributedDamage': 2428840, 'contributedBr...","[[{'contributedDamage': 1903033, 'contributedB...","[{'contributedDamage': 126637, 'contributedBre..."
1,2,Brando.1374,Specter,34,Xxkakarot,0,10,0,0,"[{'contributedDamage': 3305345, 'contributedBr...","[[{'contributedDamage': 2766494, 'contributedB...","[{'contributedDamage': 87735, 'contributedBrea..."
2,2,lPRATESl.6931,Firebrand,45,Headshot Free,10,0,10,10,"[{'contributedDamage': 429369, 'contributedBre...","[[{'contributedDamage': 329231, 'contributedBr...","[{'contributedDamage': 119269, 'contributedBre..."
3,2,Andarck.5610,Renegade,51,Saiku Goldmane,0,10,0,0,"[{'contributedDamage': 1833554, 'contributedBr...","[[{'contributedDamage': 1632661, 'contributedB...","[{'contributedDamage': 99459, 'contributedBrea..."
4,2,The Optional.1538,Scourge,86,Deutschlad,0,10,0,0,"[{'contributedDamage': 2970172, 'contributedBr...","[[{'contributedDamage': 2465464, 'contributedB...","[{'contributedDamage': 82137, 'contributedBrea..."
5,3,Saiki.5147,Druid,3,Isadora Sunspear,0,0,10,10,"[{'contributedDamage': 379429, 'contributedBre...","[[{'contributedDamage': 244034, 'contributedBr...","[{'contributedDamage': 102831, 'contributedBre..."
6,3,Yum.8039,Mechanist,52,F L O R I Ä N,0,0,0,0,"[{'contributedDamage': 2881217, 'contributedBr...","[[{'contributedDamage': 2384872, 'contributedB...","[{'contributedDamage': 102069, 'contributedBre..."
7,3,skyrim.9602,Mechanist,55,Exxcelsiorr,0,0,0,0,"[{'contributedDamage': 2672321, 'contributedBr...","[[{'contributedDamage': 2203397, 'contributedB...","[{'contributedDamage': 115611, 'contributedBre..."
8,3,Nanacio.1602,Firebrand,84,Nanacia,0,10,0,0,"[{'contributedDamage': 2755294, 'contributedBr...","[[{'contributedDamage': 2269132, 'contributedB...","[{'contributedDamage': 89030, 'contributedBrea..."
9,3,Carvalho.6382,Firebrand,89,Carvalhoooo,0,10,0,0,"[{'contributedDamage': 2538895, 'contributedBr...","[[{'contributedDamage': 1991858, 'contributedB...","[{'contributedDamage': 99605, 'contributedBrea..."


In [32]:
players_info = df.loc[:,['group','acc','profession','name', 'tough', 'condi', 'conc', 'heal']]
players_info

,group,acc,profession,name,tough,condi,conc,heal
0,2,Lemos.8360,Scourge,Paiani,0,10,0,0
1,2,Brando.1374,Specter,Xxkakarot,0,10,0,0
2,2,lPRATESl.6931,Firebrand,Headshot Free,10,0,10,10
3,2,Andarck.5610,Renegade,Saiku Goldmane,0,10,0,0
4,2,The Optional.1538,Scourge,Deutschlad,0,10,0,0
5,3,Saiki.5147,Druid,Isadora Sunspear,0,0,10,10
6,3,Yum.8039,Mechanist,F L O R I Ä N,0,0,0,0
7,3,skyrim.9602,Mechanist,Exxcelsiorr,0,0,0,0
8,3,Nanacio.1602,Firebrand,Nanacia,0,10,0,0
9,3,Carvalho.6382,Firebrand,Carvalhoooo,0,10,0,0


In [34]:
#players_info.to_csv('players_info.csv', index=False)